In [25]:
answers = []

In [15]:
import pickle
import pandas as pd

type = 'yellow'
year = 2023
month = 3

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{type}_tripdata_{year:04d}-{month:02d}.parquet'

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

df = read_data(input_file)

dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [17]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [19]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['duration'] = y_pred

In [20]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [27]:
y_pred.std()

6.247488852238703

In [24]:
!ls -lah output/

total 66M
drwxrwxr-x  2 eugene eugene 4.0K Jul 26 10:41 .
drwxrwxr-x 19 eugene eugene  12K Jul 26 10:41 ..
-rw-rw-r--  1 eugene eugene  66M Jul 26 10:24 yellow_tripdata_2023-03.parquet


In [21]:
!jupyter nbconvert --to script hw04.ipynb

[NbConvertApp] Converting notebook hw04.ipynb to script
[NbConvertApp] Writing 1267 bytes to hw04.py


In [22]:
!cat Pipfile.lock | grep scikit-learn -A 2 | tail -n 1

                "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c",


In [28]:
!python hw04.py 2023 4

14.292282936862449


In [30]:
!podman run -e TAXI_YEAR=2023 -e TAXI_MONTH=05 hw04:v1

0.19174419265916945
